<a href="https://colab.research.google.com/github/albertometelli/rl-phd-2022/blob/main/02_gym_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Gym Environment

This notebook is inspired to the Stable Baselines3 tutorial available at [https://github.com/araffin/rl-tutorial-jnrr19](https://github.com/araffin/rl-tutorial-jnrr19).


## Introduction

In this notebook, we will learn how to build a customized environment with **Open AI Gym**.

### Links

Open AI Gym Github: [https://github.com/openai/gym](https://github.com/openai/gym)

Open AI Gym Documentation: [https://www.gymlibrary.ml](https://www.gymlibrary.ml)

Stable Baselines 3 Github:[https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)

Stable Baseline 3 Documentation: [https://stable-baselines3.readthedocs.io/en/master/](https://stable-baselines3.readthedocs.io/en/master/)

## Install Dependencies and Stable Baselines3 Using Pip

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra]

In [ ]:
import stable_baselines3
stable_baselines3.__version__

## Evaluation

A helper function to evaluate policies.

In [ ]:
def evaluate(env, policy, gamma=1., num_episodes=100):
    """
    Evaluate a RL agent
    :param env: (Env object) the Gym environment
    :param policy: (BasePolicy object) the policy in stable_baselines3
    :param gamma: (float) the discount factor
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    discounter = 1.
    all_episode_rewards = []
    for i in range(num_episodes): # iterate over the episodes
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done: # iterate over the steps until termination
            action, _ = policy.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward * discounter) # compute discounted reward
            discounter *= gamma

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    std_episode_reward = np.std(all_episode_rewards) / np.sqrt(num_episodes - 1)
    print("Mean reward:", mean_episode_reward, 
          "Std reward:", std_episode_reward,
          "Num episodes:", num_episodes)

    return mean_episode_reward, std_episode_reward

## Plotting

A helper function to plot the learning curves.

In [ ]:
import matplotlib.pyplot as plt


def plot_results(results):
    plt.figure()
    
    for k in results.keys():
        data = np.load(results[k] + '/evaluations.npz')
        ts = data['timesteps']
        res = data['results']
        _mean, _std = res.mean(axis=1), res.std(axis=1)

        plt.plot(ts, _mean, label=k)
        plt.fill_between(ts, _mean-_std, _mean+_std, alpha=.2)
        
    plt.xlabel('Timesteps')
    plt.ylabel('Average return')
    plt.legend(loc='lower right')
    
    plt.show()

## The Minigolf Environment

The `Minigolf` environment models a simple problem in which the agent has to hit a ball on a green using a putter in order to reach the hole with the minimum amount of moves. 

* The green is characterized by a **friction** $f$ that is selected uniformly random at the beginning of each episode in the interval `[0.065, 0.196]` and does not change during the episode. 
* The **position** of the ball is represented by a unidimensional variable $x_t$ that is initialized uniformly random in the interval `[1,20]`. The observation is made of the pair $s_t = (x_t,f)$. 
* The **action** $a_t$ is the force applied to the putter and has to be bounded in the interval `[1e-5,5]`. Before being applied the action is subject to a Gaussian noise, so that the actual action $u_t$ applied is given by:

$$
u_t = a_t + \epsilon \qquad \text{where} \qquad \epsilon \sim \mathcal{N}(0,\sigma^2),
$$
where $\sigma =0.1$. The movement of the ball is governed by the kinematic law:

$$
x_{t+1} = x_{t} - v_t \tau_t + \frac{1}{2} d \tau_t^2
$$

where:
* $v_t$ is the velocity computed as $v_t = u_t l$,
* $d$ is the deceleration computed as $d = \frac{5}{7} fg$,
* $\tau_t$ is the time interval computed as $\tau_t = \frac{v_t}{d}$. 

The remaining constants are the putter length $l = 1$ and the gravitational acceleration $g=9.81$. The **episode** terminates when the next state is such that the ball enters or surpasses (without entering) the hole. The **reward** is `-1` at every step and `-100` if the ball surpasses the hole. To check whether the ball will not reach, enter, or surpass the hole, refer to the following condition:

\begin{align*}
&v_t < v_{\min} \implies \text{ball does not reach the hole} \\
&v_t > v_{\max} \implies \text{ball surpasses the hole} \\
&\text{otherwise} \implies \text{ball enters the hole}
\end{align*}

where

\begin{align*}
& v_\min = \sqrt{\frac{10}{7} fgx_t}
& v_\max = \sqrt{ \frac{g(2 h - \rho)^2}{2r} + v_\min^2},
\end{align*}
where $h = 0.1$ is the hole size and $\rho = 0.02135$ is the ball radius.


**References**

Penner, A. R. "The physics of putting." Canadian Journal of Physics 80.2 (2002): 83-96.

## Exercise 1

Complete the constructor `__init__`, methods `reset` and `step` based on the environment description provided above.

In [ ]:
import gym
import numpy as np
from gym.spaces import Box

class Minigolf(gym.Env):
    """
    The Minigolf problem.
    
    """

    def __init__(self):
        super(Minigolf, self).__init__()
        
        # Constants
        self.min_pos, self.max_pos = 1.0, 20.0
        self.min_action, self.max_action = 1e-5, 5.0
        self.min_friction, self.max_friction = 0.065, 0.196
        self.putter_length = 1.0
        self.hole_size = 0.10
        self.sigma_noise = 0.1
        self.ball_radius = 0.02135
        
        
        self.action_space = # TODO
        self.observation_space = # TODO
                                                    
            
    def step(self, action):
        
        #Retrieve the state components
        x, friction = self.state

        # Clip the action within the allowed range
        action = np.clip(action, self.min_action, self.max_action)

        # TODO Add noise to the action
        # TODO Compute the speed

        # Compute the speed limits
        v_min = np.sqrt(10 / 7 * friction * 9.81 * x)
        v_max = np.sqrt((2 * self.hole_size - self.ball_radius) ** 2 \
                        * (9.81 / (2 * self.ball_radius)) + v_min ** 2)

        # TODO Compute the deceleration
        # TODO Compute the time interval
        # TODO Update the position
        
        # Clip the position
        x = np.clip(x, self.min_pos, self.max_pos)

        # TODO Compute the reward and episode termination (done)
            
        self.state = np.array([x, friction]).astype(np.float32)

        return self.state, reward, done, {}


    def reset(self):
        
        x = friction = 0.

        # TODO Random generation of initial position and friction

        self.state = np.array([x, friction]).astype(np.float32)
        
        return self.state

To be able to instance the environment with `gym.make`, we need to register the environment

In [ ]:
from gym.envs.registration import register

# Just in case you re-run the code
if "Minigolf-v1" in gym.envs.registry.env_specs:
    del gym.envs.registry.env_specs["Minigolf-v1"]

register(
    id="Minigolf-v1",
    entry_point="__main__:Minigolf",
    max_episode_steps=20,
    reward_threshold=0,
)

### Validate the environment

Stable Baselines3 provides a [helper](https://stable-baselines3.readthedocs.io/en/master/common/env_checker.html) to check that our environment complies with the Gym interface.

In [ ]:
from stable_baselines3.common.env_checker import check_env

env = Minigolf()

# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)

## Evaluate some simple Policies

* **Do-nothing policy**: a policy plays the zero action.

$$
\pi(s) = 0
$$


* **Max-action policy**: a policy that plays the maximum available actions.

$$
\pi(s) = +\infty
$$


* **Zero-mean Gaussian policy**: a policy that selects the action sampled from a Gaussian policy with zero mean and variance $\sigma^2=1$

$$
\pi(a|s) = \mathcal{N}(0,\sigma^2)
$$

In [ ]:
class DoNothingPolicy():
    
    def predict(self, obs):
        return 0, obs


class MaxActionPolicy():
    
    def predict(self, obs):
        return np.inf, obs
    

class ZeroMeanGaussianPolicy():
    
    def predict(self, obs):
        return np.random.randn(), obs

In [ ]:
env = gym.make("Minigolf-v1")

do_nothing_policy = DoNothingPolicy()

max_action_policy = MaxActionPolicy()

gauss_policy = ZeroMeanGaussianPolicy()


do_nothing_mean, do_nothing_std = evaluate(env, do_nothing_policy)
max_action_mean, max_action_std = evaluate(env, max_action_policy)
gauss_policy_mean, gauss_policy_std = evaluate(env, gauss_policy)

## Train PPO, DDPG, and SAC

We now train three algorithms suitable for environments with continuous actions: [Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html), [Deep Deterministic Policy Gradient](https://stable-baselines3.readthedocs.io/en/master/modules/ddpg.html), and [Soft Actor Critic](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html).

In [ ]:
from stable_baselines3 import PPO, DDPG, SAC


# Separate evaluation env
eval_env = gym.make('Minigolf-v1')

ppo = PPO("MlpPolicy", env, verbose=1, policy_kwargs=dict(net_arch=[32]))
ddpg = DDPG("MlpPolicy", env, verbose=1, policy_kwargs=dict(net_arch=[32]))
sac = SAC("MlpPolicy", env, verbose=1, policy_kwargs=dict(net_arch=[32]))

print('PPO')
ppo.learn(total_timesteps=50000, eval_env=eval_env, 
          eval_log_path='./logs/minigolf/ppo', eval_freq=2048, log_interval=4)

print('DDPG')
ddpg.learn(total_timesteps=50000, eval_env=eval_env, 
          eval_log_path='./logs/minigolf/ddpg', eval_freq=2048, log_interval=1024)

print('SAC')
sac.learn(total_timesteps=50000, eval_env=eval_env, 
          eval_log_path='./logs/minigolf/sac', eval_freq=2048, log_interval=2048)

We plot the learning curves.

In [ ]:
results = {'PPO': './logs/minigolf/ppo',
           'DDPG': './logs/minigolf/ddpg',
           'SAC': './logs/minigolf/sac'}
        
plot_results(results)